In [ ]:
!rm -rf sample_data

# imports

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer

In [ ]:
import torch

def topk_accuracy(similarities, labels, k = 1) -> float:
    topk_values, topk_indices = torch.topk(similarities, k=k, dim=1)
    top_predicts = topk_indices + 1
    matches = torch.any(top_predicts == labels.unsqueeze(1), dim=1)
    topk_acc = matches.float().mean().item()
    return topk_acc


# key variables

In [ ]:
! hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `colab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab`


In [ ]:
sheet_path = '/content/Ka-ChatBot_BenchMark.xlsx'
# models = ['PartAI/Tooka-SBERT-V2-Large', 'xmanii/maux-gte-persian', 'BAAI/bge-m3']
model_id = 'PartAI/Tooka-SBERT-V2-Large'


# load and prepare data

In [ ]:

faq_df = pd.read_excel(sheet_path, sheet_name="faq", index_col='idx')
samples_df = pd.read_excel(sheet_path, sheet_name="samples")

In [ ]:
faq = faq_df['faq'].tolist()
samples = samples_df['sample'].tolist()
labels = torch.tensor(samples_df['gt_idx'].tolist())
categories = faq_df['category'].tolist()

# experiment candidate models



In [ ]:
model = SentenceTransformer(model_id, trust_remote_code=True)
faq_emb = model.encode(faq)
samples_emb = model.encode(samples)
similarities = model.similarity(samples_emb, faq_emb)
predicts = torch.max(similarities, axis=1)[1] + 1
acc = ((predicts == labels).sum() / labels.shape[0]).item()
print('Accuracy:', acc)


Accuracy: 0.7526881694793701


In [ ]:
for i in [1, 2,3,5,10]:
  print(f"Top-{i} Accuracy: {topk_accuracy(similarities, labels, i)}")

Top-1 Accuracy: 0.7526881694793701
Top-2 Accuracy: 0.8817204236984253
Top-3 Accuracy: 0.9139785170555115
Top-5 Accuracy: 0.9354838728904724
Top-10 Accuracy: 0.9569892287254333


In [ ]:
top_values, top_indices = torch.topk(similarities, k=2, dim=1)

margins = top_values[:,0] - top_values[:,1]

correct_mask = (predicts == labels)
wrong_mask = (predicts != labels)

avg_margin_correct = margins[correct_mask].mean().item()
avg_margin_wrong = margins[wrong_mask].mean().item()

print(f"Avg margin (correct predictions): {avg_margin_correct:.4f}")
print(f"Avg margin (wrong predictions): {avg_margin_wrong:.4f}")


Avg margin (correct predictions): 0.1038
Avg margin (wrong predictions): 0.0324


In [ ]:
true_categories = 0
for i in range(len(labels)):
  if categories[labels[i] - 1] == categories[predicts[i] - 1]:
    true_categories += 1
print(f"Accuracy of Predicting in Correct Category: {true_categories / len(labels)}")

Accuracy of Predicting in Correct Category: 0.8279569892473119


In [ ]:
for i, (pred, label) in enumerate(zip(predicts, labels)):
    if pred != label:
        print(f"Sample {i}: True={label.item()}, Pred={pred.item()}")
        print("Query:", samples[i])
        print("Top predicted FAQ:", faq[pred.item() - 1])
        print("True FAQ:", faq[label.item()-1])
        print("---")
